<a href="https://colab.research.google.com/github/IndraniMandal/New-Revisions/blob/main/kNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# k-NN Classification

k-NN: **k** **N**earest **N**eighbors

Neighbors-based classification is a type of *instance-based learning*: it does not attempt to construct a general internal model, but simply stores instances of the training data. 
Classification is computed from a simple majority vote of the *nearest neighbors of each point*: a query point is assigned the data class which has the most representatives within the nearest neighbors of the point.


<img src = "https://drive.google.com/uc?id=1UzA4phWk09Tc0NbFf1SUlq7vdEie-yS7" width=500>

We want to assign the sample (green puzzle piece) either to the class of cats, bees, chicken, dragonflies or to the class of green worms,

* If k = 1 (orange circle) it is assigned to the class of cats inside the orange circle. 

* If k = 4 (green circle) it is assigned to the class of cats again (4 cats vs. 1 chicken inside the green circle).

## Exercise

Consider the following figure,

<img src="https://raw.githubusercontent.com/IndraniMandal/ds-assets/main/assets/knn.png" height="256" width="280">

What is the color of sample (green bullet) when we compare the sample to:

* If k = 3 neighbors?

* If k = 5 neighbors?

## k-NN Classification

<img src = "https://drive.google.com/uc?id=1MY8rTU05s-NGB3dbdayjA49oRBKGLTgk" width=700>


k-NN classification is a supervised learning algorithm, therefore the training examples are vectors in a multidimensional feature space, each with a class label. 
The training phase of the algorithm consists only of storing the feature vectors and class labels of the training samples.
In the classification phase, k is a user-defined constant, and an unlabeled vector (a query or test point) is classified by assigning the label which is most frequent among the k training samples nearest to that query point.
A commonly used distance metric for continuous variables is the Euclidean distance. For discrete variables, such as for text classification, another metric can be used, such as the Hamming distance.


## A Code Example

Let's build an k-NN classifier for the iris dataset.

In [1]:
# basic data routines
import pandas as pd

#splitting the data
from sklearn.model_selection import KFold,train_test_split

# models
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# model evaluation routines
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

##Confidence interval

In [2]:
# compute 95% confidence intervals for classification and regression
# problems

def classification_confint(acc, n):
    '''
    Compute the 95% confidence interval for a classification problem.
      acc -- classification accuracy
      n   -- number of observations used to compute the accuracy
    Returns a tuple (lb,ub)
    '''
    import math
    interval = 1.96*math.sqrt(acc*(1-acc)/n)
    lb = max(0, acc - interval)
    ub = min(1.0, acc + interval)
    return (lb,ub)

## Load and split data

In [3]:
# get data
url = 'https://raw.githubusercontent.com/IndraniMandal/ds-assets/main/assets/wdbc.csv' # the URL
df = pd.read_csv(url)
features  = df.drop(['ID','Diagnosis'],axis=1)
target = df['Diagnosis']

# do train and test
X_train, X_test, y_train, y_test = train_test_split(features, target, train_size=0.8, test_size=0.2, random_state=3)
# set up the model with k=3
model = KNeighborsClassifier(n_neighbors=3)

In [4]:
# some basic data stats
print("Shape: {}".format(df.shape))
print("Value Counts on the 'Diagnosis' Field:")
print(df['Diagnosis'].value_counts())

Shape: (569, 32)
Value Counts on the 'Diagnosis' Field:
B    357
M    212
Name: Diagnosis, dtype: int64


##Train model using k-NN

In [5]:
#train the model
model.fit(X_train, y_train)
predict_y = model.predict(X_test)

#test the model accuracy
acc = accuracy_score(y_test, predict_y)
lb, ub = classification_confint(acc, X_test.shape[0])
print("Accuracy: {:3.2f} ({:3.2f}, {:3.2f})".format(acc, lb, ub))

Accuracy: 0.93 (0.88, 0.98)


The performance is not bad for a randomly chosen value for k.  

# Model Comparison

Here we are a little bit more careful with our model construction and do a cross-validated grid search for the optimal value of k.
Furthermore we want to see how our optimal k-NN classifier performance stacks up to the performance of a decision tree model in a statistical valid manner.


Let’s work our way through this comparison using the `wdbc` dataset:

* Build optimal k-NN and tree models using grid search
* Compute the accuracy for the classifiers
* Print out the confusion matrix for each classifier
* Print out the confidence interval for each classifier
* Decide if the difference between classifiers is statistically significant or not.

## k-NN Classifier

First up is the k-NN classifier.  In order to find the optimal model we set up a grid search over the number of neighbors.  In this case we search the values from 1 to 25.

In [6]:
# KNN
model = KNeighborsClassifier()

# do the 5-fold cross validation and shuffle the data
cv = KFold(n_splits=5,  shuffle = True)

# grid search
param_grid = {'n_neighbors': list(range(1,26))}
grid = GridSearchCV(model, param_grid, cv=cv)

# performing grid search
grid.fit(X_train, y_train)
print("Grid Search: best parameters: {}".format(grid.best_params_))

# accuracy of best model with confidence interval
pred_test = grid.best_estimator_.predict(X_test)
acc = accuracy_score(y_test, pred_test)
lb,ub = classification_confint(acc,X_test.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

Grid Search: best parameters: {'n_neighbors': 5}
Accuracy: 0.92 (0.87,0.97)


In [7]:
# build the confusion matrix
labels = list(target.unique())
cm = confusion_matrix(y_true= y_test,y_pred= pred_test, labels=labels)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print("Confusion Matrix:\n{}".format(cm_df))

Confusion Matrix:
    M   B
M  35   5
B   4  70


In [8]:
y_test.value_counts()

B    74
M    40
Name: Diagnosis, dtype: int64

In [12]:
5/40

0.125

Let's take a look at the performance data.  In terms of accuracy we see that the best k-NN model has an accuracy of 92% with a confidence interval of (87%, 97%).  From a medical application perspective the confusion matrix is worrisome.  We see that of the 40 malignant test samples the model misclassifies 5 as benign.  This kind of error is called the 'false negative' error and in this case would mean that over 12.5% of the malignant cases remain undetected. We also see that of the 74 benign test samples it misclassifies 4 as malignant.  The is called the 'false positive' error. From a medical point of view this is not as worrisome because additional tests will identify these cases correctly as benign.

## Decision Trees

For decision trees we set up a grid search over the tree depth from 1 to 20 and the criterion which searches over `entropy` and `gini`.

In [32]:
# decision trees
model = DecisionTreeClassifier()

# grid search
param_grid = {'max_depth': list(range(1,21)), 'criterion': ['entropy','gini'] }
grid = GridSearchCV(model, param_grid, cv=cv)

# performing grid search
grid.fit(X_train, y_train)
print("Grid Search: best parameters: {}".format(grid.best_params_))

# accuracy of best model with confidence interval
pred_test = grid.best_estimator_.predict(X_test)
acc = accuracy_score(y_test, pred_test)
lb,ub = classification_confint(acc,X_test.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

# build the confusion matrix
labels = list(target.unique())
cm = confusion_matrix(y_true= y_test,y_pred= pred_test, labels=labels)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print("Confusion Matrix:\n{}".format(cm_df))

Grid Search: best parameters: {'criterion': 'gini', 'max_depth': 14}
Accuracy: 0.91 (0.86,0.96)
Confusion Matrix:
    M   B
M  36   4
B   6  68


In [28]:
tn, fp, fn, tp = confusion_matrix(y_true= y_test,y_pred= pred_test).ravel()
(tn, fp, fn, tp)

(67, 7, 4, 36)

In [29]:
4/40

0.1

The performance of the decision tree model is much better overall and from a medical point specifically.  Less than 10% of the malignant cases are classified as a 'false negative' giving much more confidence in its applicability in a medical setting. The accuracy of the model is 96% with a confidence interval of (86%, 96%).

## Performance Comparison and Model Selection

If we compare models we have to look beyond the raw performance numbers in this case 92% and 96% for k-NN and the decision tree model, respectively. We have to ask if the difference in performance between these two models is statistically significant.  Consider the performance of the k-NN model with an accuracy and confidence interval of,
```
92% (87%, 97%)
```
Also consider the performance of the decision tree model with an accuracy and confidence interval of,
```
91% (86% ,96%)
```
Here we see that 
the confidence intervals for the decision tree and the K-NN classifier **overlap**.  Although the performance difference between the two models is not statistically significant but here the decision tree is truly the better model because the model misclassifies few true malignant cases. Therefore we will select the decision tree model as a model for our breast cancer data.
